## Deliverable 2. Create a Customer Travel Destinations Map.

In [2]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [5]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Chapais,CA,49.7834,-74.8492,56.98,96,100,2.59,overcast clouds
1,1,Hermanus,ZA,-34.4187,19.2345,56.80,53,15,7.36,few clouds
2,2,Bluff,NZ,-46.6000,168.3333,50.81,73,8,8.57,clear sky
3,3,Tvoroyri,FO,61.5556,-6.8111,50.45,92,90,15.17,overcast clouds
4,4,Kaitangata,NZ,-46.2817,169.8464,53.71,66,0,3.53,clear sky


In [7]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 78
What is the maximum temperature you would like for your trip? 92


In [8]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
9,9,Butaritari,KI,3.0707,172.7902,81.55,74,78,13.13,broken clouds
10,10,Ishigaki,JP,24.3448,124.1572,86.13,84,75,18.41,broken clouds
25,25,The Valley,AI,18.2170,-63.0578,82.54,78,20,11.50,few clouds
29,29,Guatire,VE,10.4762,-66.5427,79.70,97,56,2.15,broken clouds
36,36,Kavaratti,IN,10.5669,72.6420,82.71,75,100,11.03,overcast clouds
37,37,Kavieng,PG,-2.5744,150.7967,83.61,67,35,4.74,scattered clouds
39,39,Tulum,MX,20.2167,-87.4667,80.55,80,94,5.75,overcast clouds
43,43,Vaini,TO,-21.2000,-175.2000,80.76,74,20,4.61,few clouds
44,44,Miraflores,MX,19.2167,-98.8083,80.02,48,100,2.84,overcast clouds
46,46,Nagua,DO,19.3832,-69.8474,86.05,74,20,17.27,few clouds


In [9]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                158
City                   158
Country                158
Lat                    158
Lng                    158
Max Temp               158
Humidity               158
Cloudiness             158
Wind Speed             158
Current Description    158
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
# No empty rows

In [11]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
9,Butaritari,KI,81.55,broken clouds,3.0707,172.7902,
10,Ishigaki,JP,86.13,broken clouds,24.3448,124.1572,
25,The Valley,AI,82.54,few clouds,18.2170,-63.0578,
29,Guatire,VE,79.70,broken clouds,10.4762,-66.5427,
36,Kavaratti,IN,82.71,overcast clouds,10.5669,72.6420,
37,Kavieng,PG,83.61,scattered clouds,-2.5744,150.7967,
39,Tulum,MX,80.55,overcast clouds,20.2167,-87.4667,
43,Vaini,TO,80.76,few clouds,-21.2000,-175.2000,
44,Miraflores,MX,80.02,overcast clouds,19.2167,-98.8083,
46,Nagua,DO,86.05,few clouds,19.3832,-69.8474,


In [14]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [27]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()
clean_hotel_df.head(20)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
9,Butaritari,KI,81.55,broken clouds,3.0707,172.7902,Isles Sunset Lodge
10,Ishigaki,JP,86.13,broken clouds,24.3448,124.1572,Hotel Peace Land Ishigakijima
25,The Valley,AI,82.54,few clouds,18.2170,-63.0578,CeBlue Villas & Beach Resort
29,Guatire,VE,79.70,broken clouds,10.4762,-66.5427,Parque Residencial Canaima I
36,Kavaratti,IN,82.71,overcast clouds,10.5669,72.6420,Hotels in Lakshadweep Islands
37,Kavieng,PG,83.61,scattered clouds,-2.5744,150.7967,Nusa Island Retreat
39,Tulum,MX,80.55,overcast clouds,20.2167,-87.4667,Kin Ha Tulum
43,Vaini,TO,80.76,few clouds,-21.2000,-175.2000,Keleti Beach Resort
44,Miraflores,MX,80.02,overcast clouds,19.2167,-98.8083,Hotel Mansión
46,Nagua,DO,86.05,few clouds,19.3832,-69.8474,Hotel Cambri


In [28]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [29]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [30]:
# 11a. Add a marker layer for each city to the map. 
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))